## MCP Server

In [1]:
import os
os.makedirs("mcpcode",exist_ok=True)

In [ ]:
!pip install fastmcp --quiet

In [30]:
%%writefile mcpcode/mcpserver1.py 

from mcp.server.fastmcp import FastMCP
import wikipedia
import requests, json

mcp = FastMCP("myMCP")

@mcp.tool()
async def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output


@mcp.tool()
async def get_wikipedia_pages(query:str)->list:
    info = wikipedia.search(query)
    return info


if __name__=="__main__":
    mcp.run(transport="http")
    

Overwriting mcpcode/mcpserver1.py


In [16]:
%%writefile mcpcode/mcpserver2.py 

from mcp.server.fastmcp import FastMCP
import wikipedia
import requests, json

mcp = FastMCP("myMCP")

@mcp.tool()
def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output


@mcp.tool()
def get_wikipedia_pages(query:str)->list:
    info = wikipedia.search(query)
    return info


if __name__=="__main__":
    mcp.run(transport="stdio")
    

Overwriting mcpcode/mcpserver2.py


In [17]:
# run the MCP server
# python mcpcode/mcpserver1.py

### Implementing an Agent which uses tools from MCP server

In [18]:
!pip install langchain_mcp_adapters mcp --quiet

In [33]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

In [34]:
client = MultiServerMCPClient({"TredenceMCP":{"url":"http://127.0.0.1:8000/mcp",
                                              "transport":"streamable_http"}})

tools = await client.get_tools()
tools

  + Exception Group Traceback (most recent call last):
  |   File "/home/user/GAAPB03-training/gen-ai/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3668, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_442698/2901729102.py", line 4, in <module>
  |     tools = await client.get_tools()
  |             ^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/user/GAAPB03-training/gen-ai/lib/python3.12/site-packages/langchain_mcp_adapters/client.py", line 137, in get_tools
  |     tools_list = await asyncio.gather(*load_mcp_tool_tasks)
  |                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/user/GAAPB03-training/gen-ai/lib/python3.12/site-packages/langchain_mcp_adapters/tools.py", line 133, in load_mcp_tools
  |     async with create_session(connection) as tool_session:
  |   File "/usr/lib/python3.12/contextlib.py", line 231, in __aexit__
  |     await self.gen.athrow(value)
  |   File "/home/user/G

In [24]:

agent = create_react_agent("azure_openai:myllm",tools)


In [25]:
await agent.ainvoke({"messages":[{"role":"user","content":"what is weather in mumbai today?"}]})

{'messages': [HumanMessage(content='what is weather in mumbai today?', additional_kwargs={}, response_metadata={}, id='c1272752-b681-42e6-ad45-6b43b5f5e736'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_r5AdDryNVoOFUQafjc18dzaZ', 'function': {'arguments': '{"city":"Mumbai"}', 'name': 'get_current_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 77, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-BmNqX2spqx6VNNtxHdDkARF4wa0WV', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': F

In [26]:
await agent.ainvoke({"messages":[{"role":"user","content":"tell me about wikipedia pages for Tata"}]})

{'messages': [HumanMessage(content='tell me about wikipedia pages for Tata', additional_kwargs={}, response_metadata={}, id='19ca2367-1956-40a7-98f2-d8ed3d8d373c'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_R2fGxBRW9cdJNHz0eLtwg1yz', 'function': {'arguments': '{"query":"Tata"}', 'name': 'get_wikipedia_pages'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 76, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-BmNqtx4u9TlO2HRZiRSWisl56N8Lz', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detecte

In [32]:
%%writefile mcpcode/mcpserver3.py 

from mcp.server.fastmcp import FastMCP
import wikipedia
import requests, json

mcp = FastMCP("myMCP")


@mcp.tool()
async def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output


@mcp.tool()
async def get_wikipedia_pages(query:str)->list:
    info = wikipedia.search(query)
    return info

import uvicorn

if __name__=="__main__":
    uvicorn.run(httpapp,host="0.0.0.0",port=8000)
    

Overwriting mcpcode/mcpserver3.py
